In [1]:
# Model comparisons

In [2]:
import cv2
from face_alignment import FaceAlignment, LandmarksType
import os
import urllib.request as urlreq

from loreal_poc.dataloaders.loaders import DataLoader300W
from loreal_poc.dataloaders.wrappers import CroppedDataLoader

from loreal_poc.models.wrappers import OpenCVWrapper, FaceAlignmentWrapper
from loreal_poc.tests.performance import NMEMean
from loreal_poc.tests.base import Test, TestDiff

In [3]:
dl = DataLoader300W(dir_path="300W/sample")
chosen_idx = 4
image, ground_truth_landmarks, _ = dl[chosen_idx]

In [4]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [5]:
facealignment_model = FaceAlignmentWrapper(model=FaceAlignment(LandmarksType.TWO_D, device=device, flip_input=False))
opencv_model = OpenCVWrapper()

In [6]:
test = Test(metric=NMEMean, threshold=1).run(model=opencv_model, dataloader=dl)

test.to_dict()

{'test_name': 'Test',
 'metric_name': 'NME_mean',
 'metric_value': 0.04136279942306024,
 'threshold': 1,
 'passed': True,
 'facial_part': 'entire face',
 'model_name': 'OpenCV',
 'dataloader_name': '300W'}

In [7]:
from loreal_poc.marks.facial_parts import FacialParts

facial_part = FacialParts.LEFT_HALF.value

test_diff = TestDiff(metric=NMEMean, threshold=1).run(
    model=opencv_model, dataloader=dl, dataloader_ref=CroppedDataLoader(dl, part=facial_part), facial_part=facial_part
)
test_diff.to_dict()

/Users/bazire/Work/loreal-poc/loreal_poc/tests/performance.py:47: RuntimeWarning: Mean of empty slice
  mes = np.nanmean(es, axis=1)


{'test_name': 'TestDiff',
 'metric_name': 'NME_mean',
 'metric_value': 0.39174846002334746,
 'threshold': 1,
 'passed': True,
 'facial_part': 'left half',
 'model_name': 'OpenCV',
 'dataloader_name': '300W'}

In [8]:
# from dataclasses import fields
def report(models, dataloader, tests, facial_parts):
    results = []
    for model in models:
        # for __facial_part in fields(FacialParts):
        #  _facial_part = __facial_part.default
        for facial_part in facial_parts:
            for test_cls in tests:
                test_result = test_cls(metric=NMEMean, threshold=1).run(
                    model=model,
                    dataloader=CroppedDataLoader(dataloader, part=facial_part),
                    dataloader_ref=dl,
                    facial_part=facial_part,
                )
                results.append(test_result.to_dict())
    return results


report = report([opencv_model], dl, [TestDiff], [FacialParts.BOTTOM_HALF.value, FacialParts.UPPER_HALF.value])

/Users/bazire/Work/loreal-poc/loreal_poc/tests/performance.py:85: RuntimeWarning: Mean of empty slice
  return np.nanmean(NMEs.get(prediction_result, marks))


In [9]:
import pandas as pd

pd.DataFrame(report)

,test_name,metric_name,metric_value,threshold,passed,facial_part,model_name,dataloader_name
0,TestDiff,NME_mean,NaN,1,False,bottom half,OpenCV,300W cropped on bottom half
1,TestDiff,NME_mean,0.040216,1,True,upper half,OpenCV,300W cropped on upper half
